In [0]:
import numpy as np

In [0]:
A = np.array([[2., 7., -8., 6.],
            [4., 4., 0., -7.],
            [-1., -3., 6., 3.],
            [9., -7., -2., -8.]])

In [0]:
f = np.array([-39., 41., 4., 113.])

### LU - разложение

In [0]:
def LU_decomposition(a):
    # Представим матрицу А в виде произведения двух матриц, A = LU, где
    # L - нижнетреугольная матрица с единицами на главной диагонали
    # U - верхнетреугольная матрица
    l = np.zeros_like(a)
    u = np.zeros_like(a)
    
    n = len(a)
    
    for j in range(n):
        u[0][j] = a[0][j]
        l[j][0] = float(a[j][0]) / float(u[0][0])
        
    for i in range(1,  n):
        for j in range(i, n):
            cur_sum = 0
            for k in range(i):
                cur_sum += l[i][k] * u[k][j]
            u[i][j] = a[i][j] - cur_sum
        for j in range(i, n):
            cur_sum = 0
            for k in range(i):
                cur_sum += l[j][k] * u[k][i]
            l[j][i] = (a[j][i] - cur_sum) / u[i][i]
    return l, u

In [5]:
A

array([[ 2.,  7., -8.,  6.],
       [ 4.,  4.,  0., -7.],
       [-1., -3.,  6.,  3.],
       [ 9., -7., -2., -8.]])

In [0]:
l, u = LU_decomposition(A)

In [7]:
l @ u

array([[ 2.,  7., -8.,  6.],
       [ 4.,  4.,  0., -7.],
       [-1., -3.,  6.,  3.],
       [ 9., -7., -2., -8.]])

In [8]:
l

array([[ 1.        ,  0.        ,  0.        ,  0.        ],
       [ 2.        ,  1.        ,  0.        ,  0.        ],
       [-0.5       , -0.05      ,  1.        ,  0.        ],
       [ 4.5       ,  3.85      , -9.85714286,  1.        ]])

In [9]:
u

array([[  2.        ,   7.        ,  -8.        ,   6.        ],
       [  0.        , -10.        ,  16.        , -19.        ],
       [  0.        ,   0.        ,   2.8       ,   5.05      ],
       [  0.        ,   0.        ,   0.        ,  87.92857143]])

In [0]:
def solution_with_LU(l, u, f):
    
    n = len(l)
    y = np.zeros(n)
    for i in range(n):
        cur_sum = 0
        for k in range(i):
            cur_sum += l[i][k] * y[k]
        y[i] = f[i] - cur_sum
    
    x = np.zeros(n)
    
    for i in range(n):
        cur_sum = 0
        for k in range(i):
            cur_sum += u[n - 1 - i][n - 1 - k] * x[n - 1 - k]
        x[n - 1 - i] = (y[n - 1 - i] - cur_sum) / u[n - 1 - i][n - 1 - i]
    return x

In [11]:
solution_with_LU(l, u, f)

array([ 8., -3.,  2., -3.])

### Метод простых итераций

In [12]:
# Выберем tau = 2/(lambda_min + lambda_max)
lambds, q = np.linalg.eig(A)
tau = 2/(np.max(lambds) + np.min(lambds))
tau

(-0.2197263533843342-0.047494776136670254j)

In [0]:
# Возьмем tau равным
tau = 0.0234

In [0]:
def metric(x, y):
    return np.sqrt(np.sum((x - y)**2))

In [0]:
def mpi(a, f, tau=0.0234, eps = 10e-6):
    E = np.diag(np.ones(len(a)))
    F = tau * f
    R = E - tau * a
    # начальное приближение
    u_k = np.zeros(len(a))
    prev = u_k
    metr = 10
    i = 0
    while (i < 100 and metr > 10e-6): #0.0001003837173332081
        #r_k = f - A * prev
        i += 1
        u_k = R @ prev + F
        #u_k = R @ prev + F
        metr = metric(u_k, prev)
        prev = u_k
        print(f'metrica = {metr}, sequences = {prev}')
    return u_k

In [18]:
tau = 0.000001
E = np.diag(np.ones(len(A)))
R = E - tau * A
print(R)
np.max(np.sum(np.abs(R), axis = 1))

[[ 9.999980e-01 -7.000000e-06  8.000000e-06 -6.000000e-06]
 [-4.000000e-06  9.999960e-01  0.000000e+00  7.000000e-06]
 [ 1.000000e-06  3.000000e-06  9.999940e-01 -3.000000e-06]
 [-9.000000e-06  7.000000e-06  2.000000e-06  1.000008e+00]]


1.000026

In [19]:
mpi(A, f, tau=0.000001)

metrica = 0.00012643970895252803, sequences = [-3.90e-05  4.10e-05  4.00e-06  1.13e-04]
metrica = 0.00012644160299452256, sequences = [-7.8000855e-05  8.2000783e-05  7.9997210e-06  2.2600155e-04]
metrica = 0.00012644349706903565, sequences = [-1.17002565e-04  1.23002349e-04  1.19991630e-05  3.39004650e-04]
metrica = 0.00012644539117606782, sequences = [-1.56005130e-04  1.64004698e-04  1.59983260e-05  4.52009300e-04]
metrica = 0.00012644728531561946, sequences = [-1.9500855e-04  2.0500783e-04  1.9997210e-05  5.6501550e-04]
metrica = 0.00012644917948769125, sequences = [-2.34012825e-04  2.46011745e-04  2.39958150e-05  6.78023251e-04]
metrica = 0.0001264510736922834, sequences = [-2.73017956e-04  2.87016443e-04  2.79941409e-05  7.91032551e-04]
metrica = 0.00012645296792939667, sequences = [-3.12023941e-04  3.28021925e-04  3.19921879e-05  9.04043401e-04]
metrica = 0.00012645486219903118, sequences = [-3.51030781e-04  3.69028189e-04  3.59899559e-05  1.01705580e-03]
metrica = 0.0001264567565

array([-0.00390423,  0.00410388,  0.00039862,  0.01130768])

### Метод Зейделя

In [0]:
from math import sqrt
import numpy as np

def seidel(A, b, eps):
    
    n = len(A)
    x = [.0 for i in range(n)]

    converge = False
    while not converge:
        x_new = np.copy(x)
        for i in range(n):
            s1 = sum(A[i][j] * x_new[j] for j in range(i))
            s2 = sum(A[i][j] * x[j] for j in range(i + 1, n))
            x_new[i] = (b[i] - s1 - s2) / A[i][i]

        converge = sqrt(sum((x_new[i] - x[i]) ** 2 for i in range(n))) <= eps
        x = x_new

    return x

In [0]:
# Для сходимости метода Зейделя
# поменяем местами строки матрицы
A = np.array([[9., -7., -2., -8.],
              [2., 7., -8., 6.],
              [-1., -3., 6., 3.],
              [4., 4., 0., -7.]])

In [0]:
f = np.array([113., -39., 4., 41.])

In [23]:
seidel(A, f, eps=10e-6)

array([ 7.99999624, -2.99999441,  2.00000449, -2.99999895])